In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
vacunacion = pd.read_excel('/content/drive/MyDrive/Datos/Alcaldia/Analitica/vacunacion/BD_inm_principal_codificacion_2.xlsx')
path = '/content/drive/MyDrive/Datos/Alcaldia/Analitica/vacunacion/04022021/Personas inmunizadas 26022021.xlsx'
inmunizadas = pd.read_excel(path) 

In [7]:
vacunacion.head()

,NUMERO AFILIADO,TIPO DOC AFILIADO,GENERO,Edad,grupo etario,Barrio,numcomuna,Nombcomuna,Comuna,LONGITUD,LATITUD,Fase vacunación,linea salud,regimen,IPS 1,IPS 2,Profesion 1,Profesion 2,PRIMER APELLIDO,SEGUNDO APELLIDO,PRIMER NOMBRE,SEGUNDO NOMBRE,PERFIL PROFESIONAL,Area COVID,Dedicación,Correo,Celular,Municipio,Fecha_Nacimiento,Departamento,DIR_RESIDENCIA,DIR_RESIDENCIA_limpiada,Acepta_inmunizacion,Autoriza_inmunizacion,Consentimiento_inmunizacion,Fecha_vacuna,Hora_vacuna,inmunizado,IPS 3,IPS 4,Profesion 3,Profesion 4,condicion_final
0,6612272,CC,M,65.0,60 a 69 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 2,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,albarracin,NaN,luis,alfonso,NaN,NaN,NaN,NaN,3.158410e+09,NaN,1955-11-14,NaN,av cl 27 47,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
1,1005339195,CC,M,20.0,20 a 29 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.136905,7.155997,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
2,63303324,CC,F,58.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
3,37930938,CC,F,55.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
4,63447746,CC,F,49.0,40 a 49 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo


In [8]:
inmunizadas

,IdPersona,DatInmTipDoc,DatInmNumDoc,DatInmPriNomPer,DatInmSegNomPer,DatInmPriApePer,DatInmSegApePer,DatInmFecNa,DatInmDir,DatInmDep,DatInmMun,DatInmTelPer,DatInmCorPer,DatInmAut,DatInmAcep,DatInmCon,DatInmAcepto,PerFecReg,PerHorReg,Genero
0,1,CC,1098752146,EDNA,ROCIO,CARRILLO,DELGADO,1993-08-01,CARRERA CAMPOHERMOSO # 3O- C BARRIO CRA 3 OCCI...,SANTANDER,BUCARAMANGA,3023715613,ednitacarrillo10@gmail.com,SI,SI,SI,SI,2021-02-18,15:38:31,F
1,2,CC,63351531,ORFENIA,NaN,CALDERON,FLOREZ,1969-07-04,DIAGONAL 61 # - DIAGONAL 61 145 A -08 OTRO F...,SANTANDER,FLORIDABLANCA,3157894080,piacal-33@hotmail.com,SI,SI,SI,SI,2021-02-18,16:18:19,F
2,3,CC,37889566,SARA,MATILDE,BARRERA,CABANZO,1964-03-05,# - CALLE 47 2 W-33 CAMPOHERMOSO BUCARAMAN...,SANTANDER,BUCARAMANGA,3008064818,sara.barrera461@gmail.com,SI,SI,SI,SI,2021-02-18,16:22:03,F
3,4,CC,37749903,ZULMI,ZULMI,PINZON,RUIZ,1980-02-29,CALLE 57 # 15- 41 OLAS II FLORIDABLANCA - SA...,SANTANDER,FLORIDABLANCA,3186319074,zumipinzonruiz@gmail.com,SI,SI,SI,SI,2021-02-18,16:42:59,F
4,5,CC,5599240,FRANCISCO,EFRAIN,VALENZUELA,MUÃ‘OZ,1958-08-02,CARRERA 29 # - 115-116 PROVENZA BUCARAMANGA ...,SANTANDER,BUCARAMANGA,3205645671,piyinvalenzuela4@gmail.com,SI,SI,SI,SI,2021-02-18,16:46:30,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,964,CC,1007142478,LEIDY,MARCELA,CARO,CARO,1997-07-09,CARRERA 20 # - 201A-18 OTRO FLORIDABLANCA - ...,SANTANDER,FLORIDABLANCA,3108649533,leidycaro09@gmail.com,SI,SI,SI,SI,2021-02-23,08:15:03,F
960,965,CC,1098747254,LAURA,JULIANA,LOPEZ,GARCIA,1993-10-03,CALLE 57 # - 42W-28 ESTORAQUES BUCARAMANGA -...,SANTANDER,BUCARAMANGA,3187740916,laura_lopez.92@hotmial.com,SI,SI,SI,SI,2021-02-23,08:21:27,F
961,966,CC,79392972,FERNANDO,NaN,BUITRAGO,CACERES,1966-08-04,CALLE 23 # - 13-28 GIRARDOT BUCARAMANGA - SA...,SANTANDER,BUCARAMANGA,3184783060,fernadobuitrago8@hotmail.com,SI,SI,SI,SI,2021-02-23,08:34:49,M
962,967,CC,79407833,HECTOR,HUGO,TORRES,WILCHES,1966-12-14,CALLE 19 # - 31-45 SAN ALONSO BUCARAMANGA - ...,SANTANDER,BUCARAMANGA,3188015187,granujo@gmail.com,SI,SI,SI,SI,2021-02-23,08:37:41,M


In [14]:
# solo seleccionar los nuevos vacunados
inmunizadas = inmunizadas.loc[~inmunizadas['DatInmNumDoc'].isin(vacunacion.loc[vacunacion['inmunizado']=='si','NUMERO AFILIADO'])]
inmunizadas

,IdPersona,DatInmTipDoc,DatInmNumDoc,DatInmPriNomPer,DatInmSegNomPer,DatInmPriApePer,DatInmSegApePer,DatInmFecNa,DatInmDir,DatInmDep,DatInmMun,DatInmTelPer,DatInmCorPer,DatInmAut,DatInmAcep,DatInmCon,DatInmAcepto,PerFecReg,PerHorReg,Genero
136,137,CE,1043876663,JAINNE,NaN,CABALLERO,ACUÃ‘A,1996-12-08,CALLE 10 # - 27-67 OTRO BUCARAMANGA - SANTANDER,SANTANDER,BUCARAMANGA,31655894420,hinata0896@hotmail.com,SI,SI,SI,SI,2021-02-19,13:21:09,F
526,529,CE,446476,RAFAEL,DE JESUS,BRICEÃ‘O,CARRERO,1995-03-16,CALLE 20 # 30- 27 SAN ALONSO BUCARAMANGA - S...,SANTANDER,BUCARAMANGA,3144760168,rbricenoc16@gmail.com,SI,SI,SI,SI,2021-02-21,07:45:33,M
625,628,CC,1140814757,NATALY,PAOLA,NEGRETTE,LOPERA,1987-12-17,CARRERA 29 # - 42-24 SOTOMAYOR BUCARAMANGA -...,SANTANDER,BUCARAMANGA,3103133624,natty712@gmail.com,SI,SI,SI,SI,2021-02-21,12:15:45,F
626,629,CC,1099369109,ESTHER,NaN,PATIÃ‘O,VARGAS,1992-11-02,CALLE 54 # 23- 10 SOTOMAYOR BUCARAMANGA - SA...,SANTANDER,BUCARAMANGA,3118069548,espava1992@gmail.com,SI,SI,SI,SI,2021-02-21,12:20:08,F
627,630,CC,9520262,LUIS,ALFONSO,PEREZ,VERA,1958-08-06,CARRERA 38 # - 45-74 CABECERA DEL LLANO BUCA...,SANTANDER,BUCARAMANGA,3164687905,lperezvera@hotmail.com,SI,SI,SI,SI,2021-02-21,12:21:17,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,964,CC,1007142478,LEIDY,MARCELA,CARO,CARO,1997-07-09,CARRERA 20 # - 201A-18 OTRO FLORIDABLANCA - ...,SANTANDER,FLORIDABLANCA,3108649533,leidycaro09@gmail.com,SI,SI,SI,SI,2021-02-23,08:15:03,F
960,965,CC,1098747254,LAURA,JULIANA,LOPEZ,GARCIA,1993-10-03,CALLE 57 # - 42W-28 ESTORAQUES BUCARAMANGA -...,SANTANDER,BUCARAMANGA,3187740916,laura_lopez.92@hotmial.com,SI,SI,SI,SI,2021-02-23,08:21:27,F
961,966,CC,79392972,FERNANDO,NaN,BUITRAGO,CACERES,1966-08-04,CALLE 23 # - 13-28 GIRARDOT BUCARAMANGA - SA...,SANTANDER,BUCARAMANGA,3184783060,fernadobuitrago8@hotmail.com,SI,SI,SI,SI,2021-02-23,08:34:49,M
962,967,CC,79407833,HECTOR,HUGO,TORRES,WILCHES,1966-12-14,CALLE 19 # - 31-45 SAN ALONSO BUCARAMANGA - ...,SANTANDER,BUCARAMANGA,3188015187,granujo@gmail.com,SI,SI,SI,SI,2021-02-23,08:37:41,M


In [16]:
# para que funcione con la version 1 del script de direcciones
inmunizadas.rename(columns={'DatInmDir':'dir_res_'}, inplace=True)
inmunizadas['COMUNA'] = np.nan
inmunizadas['DIR'] = np.nan
inmunizadas['NOMCOMUNA'] = np.nan
inmunizadas['NUMERO COMUNA'] = np.nan
inmunizadas['_bar_ver_'] = np.nan
inmunizadas['bar_ver_'] = np.nan

inmunizadas.to_excel('inmunizados.xlsx',index=False, sheet_name='Hoja1')

## Cargar el repo de geoposición

In [17]:
!git clone https://github.com/Oficina-TIC-BGA/AutomatizacionCOVID.git

Cloning into 'AutomatizacionCOVID'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 229 (delta 50), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (229/229), 4.56 MiB | 6.24 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [18]:
%%writefile paths.txt
[paths]
path_direcciones =/content/inmunizados.xlsx
path_poligonos_bucaramanga =/content/AutomatizacionCOVID/files/Barrio_Vereda_Asentamiento_shps_bucaramanga_act_1_09.csv
path_poligonos_floridablanca =/content/AutomatizacionCOVID/files/Barrio_Vereda_Comuna_floridablanca_shps.csv 
path_poligonos_giron =
path_poligonos_piedecuesta =
path_division_politica_AMB =/content/AutomatizacionCOVID/files/BARRIOS Y COMUNAS OFIC PLANEACION.xlsx

Writing paths.txt


In [19]:
!python /content/AutomatizacionCOVID/scripts/OTICFinder_run.py paths.txt

Limpiando direcciónes ....
Limpiado de direcciones finalizado ...
Geoposicionando ...
100% 341/341 [06:24<00:00,  1.13s/it]
Cruzando información de polígonos (barrios)...
Points: 100% 341/341 [00:17<00:00, 19.87it/s]
Buscando (comunas)...
Comunas: 100% 341/341 [00:00<00:00, 1841.51it/s]
Creando estructura final ...
procedimiento finalizado .... Nos vemos mañana


In [ ]:
## cargar el archivo y llevar a la estructura principal

In [22]:
inmunizadas_final = pd.read_excel('/content/inmunizados_estructura_final.xlsx')
# solo por ser primera version del script
del inmunizadas_final['bar_ver_']  
del inmunizadas_final['ciudad_geo'] 
inmunizadas_final.rename(columns={'_bar_ver_':'Barrio',
                                  'COMUNA':'Comuna',
                                  'NOMCOMUNA':'Nombcomuna',
                                  'NUMERO COMUNA':'numcomuna',
                                  'longitud':'LONGITUD',
                                  'latitud':'LATITUD',
                                  'DatInmPriNomPer':'PRIMER NOMBRE',
                                  'DatInmSegNomPer':'SEGUNDO NOMBRE',
                                  'DatInmPriApePer':'PRIMER APELLIDO',
                                  'DatInmSegApePer':'SEGUNDO APELLIDO',
                                  'Genero':'GENERO',
                                  'DatInmNumDoc':'NUMERO AFILIADO',
                                  'DatInmTipDoc':'TIPO DOC AFILIADO',
                                  'dir_res_':'DIR_RESIDENCIA',
                                  'DIR':'DIR_RESIDENCIA_limpiada',
                                  'PerFecReg':'Fecha_vacuna',
                                  'PerHorReg':'Hora_vacuna',
                                  'DatInmFecNa':'Fecha_Nacimiento',
                                  'DatInmTelPer':'Celular',
                                  'DatInmCorPer':'Correo',
                                  'DatInmMun':'Municipio',
                                  'DatInmDep':'Departamento',
                                  'DatInmAcepto':'Acepta_inmunizacion',
                                  'DatInmAut':'Autoriza_inmunizacion',
                                  'DatInmCon':'Consentimiento_inmunizacion'}, inplace=True)

inmunizadas_final

,IdPersona,TIPO DOC AFILIADO,NUMERO AFILIADO,PRIMER NOMBRE,SEGUNDO NOMBRE,PRIMER APELLIDO,SEGUNDO APELLIDO,Fecha_Nacimiento,DIR_RESIDENCIA,Departamento,Municipio,Celular,Correo,Autoriza_inmunizacion,DatInmAcep,Consentimiento_inmunizacion,Acepta_inmunizacion,Fecha_vacuna,Hora_vacuna,GENERO,Comuna,DIR_RESIDENCIA_limpiada,Nombcomuna,numcomuna,Barrio,LONGITUD,LATITUD
0,137,CE,1043876663,JAINNE,NaN,CABALLERO,ACUÃ‘A,1996-12-08,CALLE 10 # - 27-67 OTRO BUCARAMANGA - SANTANDER,SANTANDER,BUCARAMANGA,31655894420,hinata0896@hotmail.com,SI,SI,SI,SI,2021-02-19,13:21:09,F,03. SAN FRANCISCO,calle 10 27 67 otro bucaramanga santander,SAN FRANCISCO,3,UNIVERSIDAD,-73.120004,7.137728
1,529,CE,446476,RAFAEL,DE JESUS,BRICEÃ‘O,CARRERO,1995-03-16,CALLE 20 # 30- 27 SAN ALONSO BUCARAMANGA - S...,SANTANDER,BUCARAMANGA,3144760168,rbricenoc16@gmail.com,SI,SI,SI,SI,2021-02-21,07:45:33,M,13. ORIENTAL,calle 20 30 27 san alonso bucaramanga santander,ORIENTAL,13,SAN ALONSO,-73.116820,7.130270
2,628,CC,1140814757,NATALY,PAOLA,NEGRETTE,LOPERA,1987-12-17,CARRERA 29 # - 42-24 SOTOMAYOR BUCARAMANGA -...,SANTANDER,BUCARAMANGA,3103133624,natty712@gmail.com,SI,SI,SI,SI,2021-02-21,12:15:45,F,12. CABECERA DEL LLANO,carrera 29 42 24 sotomayor bucaramanga santander,CABECERA DEL LLANO,12,SOTOMAYOR,-73.113040,7.119970
3,629,CC,1099369109,ESTHER,NaN,PATIÃ‘O,VARGAS,1992-11-02,CALLE 54 # 23- 10 SOTOMAYOR BUCARAMANGA - SA...,SANTANDER,BUCARAMANGA,3118069548,espava1992@gmail.com,SI,SI,SI,SI,2021-02-21,12:20:08,F,12. CABECERA DEL LLANO,calle 54 23 10 sotomayor bucaramanga santander,CABECERA DEL LLANO,12,BOLARQUI,-73.115420,7.110870
4,630,CC,9520262,LUIS,ALFONSO,PEREZ,VERA,1958-08-06,CARRERA 38 # - 45-74 CABECERA DEL LLANO BUCA...,SANTANDER,BUCARAMANGA,3164687905,lperezvera@hotmail.com,SI,SI,SI,SI,2021-02-21,12:21:17,M,12. CABECERA DEL LLANO,carrera 38 45 74 cabecera del llano bucaramang...,CABECERA DEL LLANO,12,CABECERA DEL LLANO,-73.108220,7.118720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,964,CC,1007142478,LEIDY,MARCELA,CARO,CARO,1997-07-09,CARRERA 20 # - 201A-18 OTRO FLORIDABLANCA - ...,SANTANDER,FLORIDABLANCA,3108649533,leidycaro09@gmail.com,SI,SI,SI,SI,2021-02-23,08:15:03,F,22. CAÑAVERAL -VERSALLES,carrera 20 201a 18 otro floridablanca santander,CAÑAVERAL -VERSALLES,22,LA PAZ,-73.096660,7.060243
337,965,CC,1098747254,LAURA,JULIANA,LOPEZ,GARCIA,1993-10-03,CALLE 57 # - 42W-28 ESTORAQUES BUCARAMANGA -...,SANTANDER,BUCARAMANGA,3187740916,laura_lopez.92@hotmial.com,SI,SI,SI,SI,2021-02-23,08:21:27,F,17. MUTIS,calle 57 42w 28 estoraques bucaramanga santander,MUTIS,17,ESTORAQUES,-73.144217,7.092564
338,966,CC,79392972,FERNANDO,NaN,BUITRAGO,CACERES,1966-08-04,CALLE 23 # - 13-28 GIRARDOT BUCARAMANGA - SA...,SANTANDER,BUCARAMANGA,3184783060,fernadobuitrago8@hotmail.com,SI,SI,SI,SI,2021-02-23,08:34:49,M,04. OCCIDENTAL,calle 23 13 28 girardot bucaramanga santander,OCCIDENTAL,4,GIRARDOT,-73.134081,7.124065
339,967,CC,79407833,HECTOR,HUGO,TORRES,WILCHES,1966-12-14,CALLE 19 # - 31-45 SAN ALONSO BUCARAMANGA - ...,SANTANDER,BUCARAMANGA,3188015187,granujo@gmail.com,SI,SI,SI,SI,2021-02-23,08:37:41,M,13. ORIENTAL,calle 19 31 45 san alonso bucaramanga santander,ORIENTAL,13,SAN ALONSO,-73.116172,7.131280


In [23]:
set(vacunacion.keys()).difference(set(inmunizadas_final.keys()))

{'Area COVID',
 'Dedicación',
 'Edad',
 'Fase vacunación',
 'IPS 1',
 'IPS 2',
 'IPS 3',
 'IPS 4',
 'PERFIL PROFESIONAL',
 'Profesion 1',
 'Profesion 2',
 'Profesion 3',
 'Profesion 4',
 'condicion_final',
 'grupo etario',
 'inmunizado',
 'linea salud',
 'regimen'}

In [24]:
set(inmunizadas_final.keys()).difference(vacunacion.keys())

{'DatInmAcep', 'IdPersona'}

In [25]:
columns = list(set(inmunizadas_final.keys()).intersection(vacunacion.keys()))
columns

['LATITUD',
 'TIPO DOC AFILIADO',
 'Autoriza_inmunizacion',
 'DIR_RESIDENCIA',
 'Municipio',
 'LONGITUD',
 'Nombcomuna',
 'PRIMER NOMBRE',
 'Correo',
 'Hora_vacuna',
 'Comuna',
 'Barrio',
 'PRIMER APELLIDO',
 'Fecha_vacuna',
 'DIR_RESIDENCIA_limpiada',
 'Acepta_inmunizacion',
 'SEGUNDO APELLIDO',
 'Departamento',
 'numcomuna',
 'NUMERO AFILIADO',
 'Consentimiento_inmunizacion',
 'SEGUNDO NOMBRE',
 'Celular',
 'Fecha_Nacimiento',
 'GENERO']

## ACTUALIZACION DATOS

In [26]:
# hacer la coincidencia por tipo y numero de documento
vacunacion['NUMERO AFILIADO'] = vacunacion['NUMERO AFILIADO'].astype(str).str.strip()
inmunizadas_final['NUMERO AFILIADO'] = inmunizadas_final['NUMERO AFILIADO'].astype(str).str.strip()
vacunacion['TIPO DOC AFILIADO'] = vacunacion['TIPO DOC AFILIADO'].astype(str).str.strip()
inmunizadas_final['TIPO DOC AFILIADO'] = inmunizadas_final['TIPO DOC AFILIADO'].astype(str).str.strip()

In [27]:
## revisar si hay repetidos en los inmunizados
obj = (inmunizadas_final.groupby(['NUMERO AFILIADO', 'TIPO DOC AFILIADO']).count()['GENERO']>1)
mask = obj.values
# cedulas que estan dos veces
cc_rep = obj[mask].index
print('repetidos: ',len(cc_rep))

repetidos:  0


In [28]:
# realizar cruces
cruce_enc = inmunizadas_final.loc[((inmunizadas_final['TIPO DOC AFILIADO'].isin(vacunacion['TIPO DOC AFILIADO'])) &
               (inmunizadas_final['NUMERO AFILIADO'].isin(vacunacion['NUMERO AFILIADO']))), columns]
cruce_no_enc =  inmunizadas_final.loc[~((inmunizadas_final['TIPO DOC AFILIADO'].isin(vacunacion['TIPO DOC AFILIADO'])) &
               (inmunizadas_final['NUMERO AFILIADO'].isin(vacunacion['NUMERO AFILIADO']))), columns]              

In [29]:
# comprobar dimensiones
assert cruce_enc.shape[0]+cruce_no_enc.shape[0] == inmunizadas_final.shape[0], 'No coinciden las dimensiones'

In [30]:
cruce_enc

,LATITUD,TIPO DOC AFILIADO,Autoriza_inmunizacion,DIR_RESIDENCIA,Municipio,LONGITUD,Nombcomuna,PRIMER NOMBRE,Correo,Hora_vacuna,Comuna,Barrio,PRIMER APELLIDO,Fecha_vacuna,DIR_RESIDENCIA_limpiada,Acepta_inmunizacion,SEGUNDO APELLIDO,Departamento,numcomuna,NUMERO AFILIADO,Consentimiento_inmunizacion,SEGUNDO NOMBRE,Celular,Fecha_Nacimiento,GENERO
0,7.137728,CE,SI,CALLE 10 # - 27-67 OTRO BUCARAMANGA - SANTANDER,BUCARAMANGA,-73.120004,SAN FRANCISCO,JAINNE,hinata0896@hotmail.com,13:21:09,03. SAN FRANCISCO,UNIVERSIDAD,CABALLERO,2021-02-19,calle 10 27 67 otro bucaramanga santander,SI,ACUÃ‘A,SANTANDER,3,1043876663,SI,NaN,31655894420,1996-12-08,F
1,7.130270,CE,SI,CALLE 20 # 30- 27 SAN ALONSO BUCARAMANGA - S...,BUCARAMANGA,-73.116820,ORIENTAL,RAFAEL,rbricenoc16@gmail.com,07:45:33,13. ORIENTAL,SAN ALONSO,BRICEÃ‘O,2021-02-21,calle 20 30 27 san alonso bucaramanga santander,SI,CARRERO,SANTANDER,13,446476,SI,DE JESUS,3144760168,1995-03-16,M
2,7.119970,CC,SI,CARRERA 29 # - 42-24 SOTOMAYOR BUCARAMANGA -...,BUCARAMANGA,-73.113040,CABECERA DEL LLANO,NATALY,natty712@gmail.com,12:15:45,12. CABECERA DEL LLANO,SOTOMAYOR,NEGRETTE,2021-02-21,carrera 29 42 24 sotomayor bucaramanga santander,SI,LOPERA,SANTANDER,12,1140814757,SI,PAOLA,3103133624,1987-12-17,F
3,7.110870,CC,SI,CALLE 54 # 23- 10 SOTOMAYOR BUCARAMANGA - SA...,BUCARAMANGA,-73.115420,CABECERA DEL LLANO,ESTHER,espava1992@gmail.com,12:20:08,12. CABECERA DEL LLANO,BOLARQUI,PATIÃ‘O,2021-02-21,calle 54 23 10 sotomayor bucaramanga santander,SI,VARGAS,SANTANDER,12,1099369109,SI,NaN,3118069548,1992-11-02,F
4,7.118720,CC,SI,CARRERA 38 # - 45-74 CABECERA DEL LLANO BUCA...,BUCARAMANGA,-73.108220,CABECERA DEL LLANO,LUIS,lperezvera@hotmail.com,12:21:17,12. CABECERA DEL LLANO,CABECERA DEL LLANO,PEREZ,2021-02-21,carrera 38 45 74 cabecera del llano bucaramang...,SI,VERA,SANTANDER,12,9520262,SI,ALFONSO,3164687905,1958-08-06,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,7.060243,CC,SI,CARRERA 20 # - 201A-18 OTRO FLORIDABLANCA - ...,FLORIDABLANCA,-73.096660,CAÑAVERAL -VERSALLES,LEIDY,leidycaro09@gmail.com,08:15:03,22. CAÑAVERAL -VERSALLES,LA PAZ,CARO,2021-02-23,carrera 20 201a 18 otro floridablanca santander,SI,CARO,SANTANDER,22,1007142478,SI,MARCELA,3108649533,1997-07-09,F
337,7.092564,CC,SI,CALLE 57 # - 42W-28 ESTORAQUES BUCARAMANGA -...,BUCARAMANGA,-73.144217,MUTIS,LAURA,laura_lopez.92@hotmial.com,08:21:27,17. MUTIS,ESTORAQUES,LOPEZ,2021-02-23,calle 57 42w 28 estoraques bucaramanga santander,SI,GARCIA,SANTANDER,17,1098747254,SI,JULIANA,3187740916,1993-10-03,F
338,7.124065,CC,SI,CALLE 23 # - 13-28 GIRARDOT BUCARAMANGA - SA...,BUCARAMANGA,-73.134081,OCCIDENTAL,FERNANDO,fernadobuitrago8@hotmail.com,08:34:49,04. OCCIDENTAL,GIRARDOT,BUITRAGO,2021-02-23,calle 23 13 28 girardot bucaramanga santander,SI,CACERES,SANTANDER,4,79392972,SI,NaN,3184783060,1966-08-04,M
339,7.131280,CC,SI,CALLE 19 # - 31-45 SAN ALONSO BUCARAMANGA - ...,BUCARAMANGA,-73.116172,ORIENTAL,HECTOR,granujo@gmail.com,08:37:41,13. ORIENTAL,SAN ALONSO,TORRES,2021-02-23,calle 19 31 45 san alonso bucaramanga santander,SI,WILCHES,SANTANDER,13,79407833,SI,HUGO,3188015187,1966-12-14,M


In [31]:
# update podria funcionar aquí también
merged = pd.merge(vacunacion, cruce_enc, on=['TIPO DOC AFILIADO', 'NUMERO AFILIADO'], how='left', suffixes=('', '_y'), validate='1:1')
merged

,NUMERO AFILIADO,TIPO DOC AFILIADO,GENERO,Edad,grupo etario,Barrio,numcomuna,Nombcomuna,Comuna,LONGITUD,LATITUD,Fase vacunación,linea salud,regimen,IPS 1,IPS 2,Profesion 1,Profesion 2,PRIMER APELLIDO,SEGUNDO APELLIDO,PRIMER NOMBRE,SEGUNDO NOMBRE,PERFIL PROFESIONAL,Area COVID,Dedicación,Correo,Celular,Municipio,Fecha_Nacimiento,Departamento,DIR_RESIDENCIA,DIR_RESIDENCIA_limpiada,Acepta_inmunizacion,Autoriza_inmunizacion,Consentimiento_inmunizacion,Fecha_vacuna,Hora_vacuna,inmunizado,IPS 3,IPS 4,Profesion 3,Profesion 4,condicion_final,LATITUD_y,Autoriza_inmunizacion_y,DIR_RESIDENCIA_y,Municipio_y,LONGITUD_y,Nombcomuna_y,PRIMER NOMBRE_y,Correo_y,Hora_vacuna_y,Comuna_y,Barrio_y,PRIMER APELLIDO_y,Fecha_vacuna_y,DIR_RESIDENCIA_limpiada_y,Acepta_inmunizacion_y,SEGUNDO APELLIDO_y,Departamento_y,numcomuna_y,Consentimiento_inmunizacion_y,SEGUNDO NOMBRE_y,Celular_y,Fecha_Nacimiento_y,GENERO_y
0,6612272,CC,M,65.0,60 a 69 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 2,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,albarracin,NaN,luis,alfonso,NaN,NaN,NaN,NaN,3.158410e+09,NaN,1955-11-14,NaN,av cl 27 47,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,1005339195,CC,M,20.0,20 a 29 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.136905,7.155997,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,63303324,CC,F,58.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,37930938,CC,F,55.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,63447746,CC,F,49.0,40 a 49 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555145,1115733438,CC,M,29.0,20 a 29 años,NaN,NaN,NaN,NaN,NaN,NaN,Etapa 2,Profesionales Salud 2 Línea,Régimen contributivo,CLINICA CHICAMOCHA S.A,DISPENSARIO MEDICO BUCARAMANGA,NaN,PROF DE SALUD ESP. EN AUDITORIA DMBUG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
555146,1122810279,CC,M,33.0,30 a 39 años,NaN,NaN,NaN,NaN,NaN,NaN,Etapa 2,Profesionales Salud 2 Línea,Régimen contributivo,CLINICADEURGENCIASBUCARAMANGASAS,CLINICA CHICAMOCHA S.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
555147,1140419302,CC,M,43.0,40 a 49 años,NaN,NaN,NaN,NaN,NaN,NaN,Etapa 1,Profesionales Salud 1 Línea,Régimen contributivo,LOS COMUNEROS HOSPITAL UNIVERSITARIO DE BUCARA...,DISPENSARIO MEDICO BUCARAMANGA,GASTROENTEROLOGÍA,MEDICO GASTROENTEROLOGO DMBUG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo,NaN,Na

In [32]:
# quitar las keys
columns.pop(columns.index('TIPO DOC AFILIADO'))
columns.pop(columns.index('NUMERO AFILIADO'))
columns

['LATITUD',
 'Autoriza_inmunizacion',
 'DIR_RESIDENCIA',
 'Municipio',
 'LONGITUD',
 'Nombcomuna',
 'PRIMER NOMBRE',
 'Correo',
 'Hora_vacuna',
 'Comuna',
 'Barrio',
 'PRIMER APELLIDO',
 'Fecha_vacuna',
 'DIR_RESIDENCIA_limpiada',
 'Acepta_inmunizacion',
 'SEGUNDO APELLIDO',
 'Departamento',
 'numcomuna',
 'Consentimiento_inmunizacion',
 'SEGUNDO NOMBRE',
 'Celular',
 'Fecha_Nacimiento',
 'GENERO']

In [33]:
# COLOCAR TODO LO DE LA DERECHA A MENOS QUE SEA NULO CON UPDATE ES AL REVEZ LA ACTUALIZACION
for c in columns:
    merged[c] = merged[c+'_y'].where(~merged[c+'_y'].isnull(), vacunacion[c])
    del merged[c+'_y']

In [34]:
merged

,NUMERO AFILIADO,TIPO DOC AFILIADO,GENERO,Edad,grupo etario,Barrio,numcomuna,Nombcomuna,Comuna,LONGITUD,LATITUD,Fase vacunación,linea salud,regimen,IPS 1,IPS 2,Profesion 1,Profesion 2,PRIMER APELLIDO,SEGUNDO APELLIDO,PRIMER NOMBRE,SEGUNDO NOMBRE,PERFIL PROFESIONAL,Area COVID,Dedicación,Correo,Celular,Municipio,Fecha_Nacimiento,Departamento,DIR_RESIDENCIA,DIR_RESIDENCIA_limpiada,Acepta_inmunizacion,Autoriza_inmunizacion,Consentimiento_inmunizacion,Fecha_vacuna,Hora_vacuna,inmunizado,IPS 3,IPS 4,Profesion 3,Profesion 4,condicion_final
0,6612272,CC,M,65.0,60 a 69 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 2,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,albarracin,NaN,luis,alfonso,NaN,NaN,NaN,NaN,3.158410e+09,NaN,1955-11-14,NaN,av cl 27 47,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
1,1005339195,CC,M,20.0,20 a 29 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.136905,7.155997,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
2,63303324,CC,F,58.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
3,37930938,CC,F,55.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
4,63447746,CC,F,49.0,40 a 49 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555145,1115733438,CC,M,29.0,20 a 29 años,NaN,NaN,NaN,NaN,NaN,NaN,Etapa 2,Profesionales Salud 2 Línea,Régimen contributivo,CLINICA CHICAMOCHA S.A,DISPENSARIO MEDICO BUCARAMANGA,NaN,PROF DE SALUD ESP. EN AUDITORIA DMBUG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
555146,1122810279,CC,M,33.0,30 a 39 años,NaN,NaN,NaN,NaN,NaN,NaN,Etapa 2,Profesionales Salud 2 Línea,Régimen contributivo,CLINICADEURGENCIASBUCARAMANGASAS,CLINICA CHICAMOCHA S.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
555147,1140419302,CC,M,43.0,40 a 49 años,NaN,NaN,NaN,NaN,NaN,NaN,Etapa 1,Profesionales Salud 1 Línea,Régimen contributivo,LOS COMUNEROS HOSPITAL UNIVERSITARIO DE BUCARA...,DISPENSARIO MEDICO BUCARAMANGA,GASTROENTEROLOGÍA,MEDICO GASTROENTEROLOGO DMBUG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
555148,91234136,CC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.116663,Etapa 1,Profesionales Salud 2 Línea,NaN,CLINICA MATERNO INFANTIL SAN LUIS SA,CLINICADEURGENCIASBUCARAMANGASAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,SERVICLINICOS DROMEDICA SA,NaN,NaN,NaN,vivo


In [35]:
## REVISAR LA RAZON DE PORQUE NO SE ENCONTRARON LOS RESTANTES
# revisar si solo por la cedula hay coincidencias, hay coincidencias? si es falso agregar como nuevos
cruce_no_enc.loc[(cruce_no_enc['NUMERO AFILIADO'].isin(vacunacion['NUMERO AFILIADO']))].shape[0]!=0

False

In [36]:
# crearles el resto de columnas e insertar
for col in list(set(vacunacion.keys()).difference(set(cruce_no_enc.keys()))):
    cruce_no_enc[col] = np.nan

In [37]:
merged = pd.concat([merged, cruce_no_enc])

In [38]:
merged

,NUMERO AFILIADO,TIPO DOC AFILIADO,GENERO,Edad,grupo etario,Barrio,numcomuna,Nombcomuna,Comuna,LONGITUD,LATITUD,Fase vacunación,linea salud,regimen,IPS 1,IPS 2,Profesion 1,Profesion 2,PRIMER APELLIDO,SEGUNDO APELLIDO,PRIMER NOMBRE,SEGUNDO NOMBRE,PERFIL PROFESIONAL,Area COVID,Dedicación,Correo,Celular,Municipio,Fecha_Nacimiento,Departamento,DIR_RESIDENCIA,DIR_RESIDENCIA_limpiada,Acepta_inmunizacion,Autoriza_inmunizacion,Consentimiento_inmunizacion,Fecha_vacuna,Hora_vacuna,inmunizado,IPS 3,IPS 4,Profesion 3,Profesion 4,condicion_final
0,6612272,CC,M,65.0,60 a 69 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 2,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,albarracin,NaN,luis,alfonso,NaN,NaN,NaN,NaN,3.158410e+09,NaN,1955-11-14,NaN,av cl 27 47,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
1,1005339195,CC,M,20.0,20 a 29 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.136905,7.155997,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
2,63303324,CC,F,58.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
3,37930938,CC,F,55.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
4,63447746,CC,F,49.0,40 a 49 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,1095922526,CC,F,NaN,NaN,GIRON,0.0,GIRON,GIRON,-73.168437,7.072507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OLARTE,MORENO,LISSEGE,MARITSSA,NaN,NaN,NaN,giselleolarte@hotmail.com,3.508792e+09,GIRON,1990-07-10,SANTANDER,CALLE 40 # - 23-81 OTRO GIRON - SANTANDER,calle 40 23 81 otro giron santander,SI,SI,SI,2021-02-22,19:32:52,NaN,NaN,NaN,NaN,NaN,NaN
285,91078177,CC,M,NaN,NaN,CAÑAVERAL,22.0,CAÑAVERAL -VERSALLES,22. CAÑAVERAL -VERSALLES,-73.106213,7.068851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QUIROZ,HERNANDEZ,JUAN,MANUEL,NaN,NaN,NaN,juanma05@hotmail.com,3.228668e+09,FLORIDABLANCA,1980-08-05,SANTANDER,CALLE 31 # - 24-68 OTRO FLORIDABLANCA - SANT...,calle 31 24 68 otro floridablanca santander,SI,SI,SI,2021-02-22,19:33:42,NaN,NaN,NaN,NaN,NaN,NaN
288,1098719934,CC,F,NaN,NaN,PALOMITAS,25.0,BOSQUES - MOLINOS,25. BOSQUES - MOLINOS,-73.112086,7.075356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASTELLANOS,BALLE,MELISSA,ALEJANDRA,NaN,NaN,NaN,melissacastellanos3007@gmail.com,3.173780e+09,FLORIDABLANCA,1992-03-30,SANTANDER,CARRERA 21 # - 152-30 PALOMITAS FLORIDABLANC...,carrera 21 152 30 palomitas floridablanca sant...,SI,SI,SI,2021-02-22,19:38:47,NaN,NaN,NaN,NaN,NaN,NaN
305,91534136,CC,M,NaN,NaN,PIEDECUESTA,0.0,PIEDECUESTA,PIEDECUESTA,-73.034030,7.047020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MACIAS,RUEDA,CRISTIAN,MAURICIO,NaN,NaN,NaN,c.maciast84@gmail.com,3.214403e+09,PIEDECUESTA,1984-11-09,SANTANDER,VEREDA LAMATA # - GRANJA OTRO PIEDECUESTA - ...,vereda lamata granja otro piedecuesta santander,SI,SI,SI,2021-02-22,20:00:47,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# 1. regla colocar vacunado
merged.loc[(~merged.Fecha_vacuna.isnull()), 'inmunizado'] = 'si'

In [40]:
merged['Fecha_Nacimiento'] = merged['Fecha_Nacimiento'].astype('datetime64[ns]')
merged['Fecha_vacuna'] = merged['Fecha_vacuna'].astype('datetime64[ns]')

In [41]:
# 2. de los que encontro con fecha de nacimiento, calcular a la fecha de vacunacion la edad
merged.loc[~(merged['Fecha_Nacimiento'].isnull()) & 
            (merged.Edad.isnull()), 'Edad'] = np.floor((merged.loc[~(merged['Fecha_Nacimiento'].isnull()) & 
                                                       (merged.Edad.isnull()), 'Fecha_vacuna'] - merged.loc[~(merged['Fecha_Nacimiento'].isnull()) & 
                                                       (merged.Edad.isnull()), 'Fecha_Nacimiento']).dt.days/365.25)

In [42]:
def to_grupo_etario(x):
    if 16 <= x <=19:
        y = '16 a 19 años'
    elif 20 <= x <= 29:
        y = '20 a 29 años'
    elif 30 <= x <= 39:
        y = '30 a 39 años'  
    elif 40 <= x <= 49:
        y = '40 a 49 años'  
    elif 50 <= x <= 59:
        y = '50 a 59 años' 
    elif 60 <= x <= 69:
        y = '60 a 69 años'    
    elif 70 <= x <= 79:
        y = '70 a 79 años'    
    elif 80 <= x <= 89:
        y = '80 a 89 años'  
    elif 90 <= x <= 99:
        y = '90 a 99 años' 
    elif 100 <= x <= 109:
        y = '100 a 109 años'
    elif x >= 110:
        y = '>110 años' 
    elif x < 16:
        y = '<16 años'
    return y                                                                                      

In [43]:
query = (merged['grupo etario'].isnull()) & ~(merged.Edad.isnull())
merged.loc[query, 'grupo etario'] = merged.loc[query, 'Edad'].apply(lambda x: to_grupo_etario(x))

In [44]:
# corroborrar el numero de vacunados
merged.loc[merged.inmunizado=='si'].shape

(960, 43)

In [ ]:
!pip install xlsxwriter

In [ ]:
merged.to_excel('/content/drive/MyDrive/Datos/Alcaldia/Analitica/vacunacion/BD_inm_principal_codificacion_2.xlsx', index=False, engine='xlsxwriter')

In [ ]:
merged.to_excel('BD_inm_principal_cod_2.xlsx', index=False, engine='xlsxwriter')

## SOLO EN EL CASO DE SI HAY FALTANTES

In [46]:
# Vacunados faltantes (en este caso son 4)
path = '/content/drive/MyDrive/Datos/Alcaldia/Analitica/vacunacion/04022021/Personas inmunizadas 26022021.xlsx'
inmunizadas_base_dia = pd.read_excel(path) 

In [52]:
faltantes = inmunizadas_base_dia.loc[~inmunizadas_base_dia['DatInmNumDoc'].astype(str).str.strip().isin(merged.loc[merged.inmunizado=='si', 'NUMERO AFILIADO'])]

In [59]:
# buscar estos a nivel general en la base
#confirmar que no esten
merged.loc[merged['NUMERO AFILIADO'].isin(faltantes['DatInmNumDoc'])].shape[0]==0

True

In [63]:
# ahora buscarlos en inmunizadas
inmunizadas_final.loc[inmunizadas_final['NUMERO AFILIADO'].isin(faltantes['DatInmNumDoc'])]

,IdPersona,TIPO DOC AFILIADO,NUMERO AFILIADO,PRIMER NOMBRE,SEGUNDO NOMBRE,PRIMER APELLIDO,SEGUNDO APELLIDO,Fecha_Nacimiento,DIR_RESIDENCIA,Departamento,Municipio,Celular,Correo,Autoriza_inmunizacion,DatInmAcep,Consentimiento_inmunizacion,Acepta_inmunizacion,Fecha_vacuna,Hora_vacuna,GENERO,Comuna,DIR_RESIDENCIA_limpiada,Nombcomuna,numcomuna,Barrio,LONGITUD,LATITUD


In [65]:
cruce_no_enc.loc[cruce_no_enc['NUMERO AFILIADO'].isin(faltantes['DatInmNumDoc'])]

,LATITUD,TIPO DOC AFILIADO,Autoriza_inmunizacion,DIR_RESIDENCIA,Municipio,LONGITUD,Nombcomuna,PRIMER NOMBRE,Correo,Hora_vacuna,Comuna,Barrio,PRIMER APELLIDO,Fecha_vacuna,DIR_RESIDENCIA_limpiada,Acepta_inmunizacion,SEGUNDO APELLIDO,Departamento,numcomuna,NUMERO AFILIADO,Consentimiento_inmunizacion,SEGUNDO NOMBRE,Celular,Fecha_Nacimiento,GENERO,Fase vacunación,IPS 1,IPS 3,condicion_final,Edad,grupo etario,Profesion 3,Profesion 2,IPS 4,regimen,inmunizado,Area COVID,linea salud,PERFIL PROFESIONAL,IPS 2,Profesion 1,Dedicación,Profesion 4


In [66]:
faltantes

,IdPersona,DatInmTipDoc,DatInmNumDoc,DatInmPriNomPer,DatInmSegNomPer,DatInmPriApePer,DatInmSegApePer,DatInmFecNa,DatInmDir,DatInmDep,DatInmMun,DatInmTelPer,DatInmCorPer,DatInmAut,DatInmAcep,DatInmCon,DatInmAcepto,PerFecReg,PerHorReg,Genero
136,137,CE,1043876663,JAINNE,NaN,CABALLERO,ACUÃ‘A,1996-12-08,CALLE 10 # - 27-67 OTRO BUCARAMANGA - SANTANDER,SANTANDER,BUCARAMANGA,31655894420,hinata0896@hotmail.com,SI,SI,SI,SI,2021-02-19,13:21:09,F
526,529,CE,446476,RAFAEL,DE JESUS,BRICEÃ‘O,CARRERO,1995-03-16,CALLE 20 # 30- 27 SAN ALONSO BUCARAMANGA - S...,SANTANDER,BUCARAMANGA,3144760168,rbricenoc16@gmail.com,SI,SI,SI,SI,2021-02-21,07:45:33,M
910,915,CC,718023,ROBERTO,CARLO,CORREA,BASTIDAS,1968-09-02,CARRERA 36 # - 37-26 APTO 604 EL PRADO BUCAR...,SANTANDER,BUCARAMANGA,3324374537,rcorrea93@gmail.com,SI,SI,SI,SI,2021-02-22,19:36:25,M
946,951,CC,476188,KATYNA,GABRIELA,DI LIZIO,MIELE,1983-07-21,CARRERA 39- # - 46-79 CABECERA DEL LLANO BUC...,SANTANDER,BUCARAMANGA,3108550959,kgdm@hotmail.com,SI,SI,SI,SI,2021-02-23,07:20:47,F


In [67]:
# para que funcione con la version 1 del script de direcciones
faltantes.rename(columns={'DatInmDir':'dir_res_'}, inplace=True)
faltantes['COMUNA'] = np.nan
faltantes['DIR'] = np.nan
faltantes['NOMCOMUNA'] = np.nan
faltantes['NUMERO COMUNA'] = np.nan
faltantes['_bar_ver_'] = np.nan
faltantes['bar_ver_'] = np.nan

faltantes.to_excel('faltantes.xlsx',index=False, sheet_name='Hoja1')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [68]:
%%writefile paths.txt
[paths]
path_direcciones =/content/faltantes.xlsx
path_poligonos_bucaramanga =/content/AutomatizacionCOVID/files/Barrio_Vereda_Asentamiento_shps_bucaramanga_act_1_09.csv
path_poligonos_floridablanca =/content/AutomatizacionCOVID/files/Barrio_Vereda_Comuna_floridablanca_shps.csv 
path_poligonos_giron =
path_poligonos_piedecuesta =
path_division_politica_AMB =/content/AutomatizacionCOVID/files/BARRIOS Y COMUNAS OFIC PLANEACION.xlsx

Overwriting paths.txt


In [69]:
!python /content/AutomatizacionCOVID/scripts/OTICFinder_run.py paths.txt

Limpiando direcciónes ....
Limpiado de direcciones finalizado ...
Geoposicionando ...
100% 4/4 [00:03<00:00,  1.17it/s]
Cruzando información de polígonos (barrios)...
Points: 100% 4/4 [00:00<00:00, 112.05it/s]
Buscando (comunas)...
Comunas: 100% 4/4 [00:00<00:00, 1379.36it/s]
Creando estructura final ...
procedimiento finalizado .... Nos vemos mañana


In [71]:
faltantes_final = pd.read_excel('/content/faltantes_estructura_final.xlsx')
# solo por ser primera version del script
del faltantes_final['bar_ver_']  
del faltantes_final['ciudad_geo'] 
faltantes_final.rename(columns={'_bar_ver_':'Barrio',
                                  'COMUNA':'Comuna',
                                  'NOMCOMUNA':'Nombcomuna',
                                  'NUMERO COMUNA':'numcomuna',
                                  'longitud':'LONGITUD',
                                  'latitud':'LATITUD',
                                  'DatInmPriNomPer':'PRIMER NOMBRE',
                                  'DatInmSegNomPer':'SEGUNDO NOMBRE',
                                  'DatInmPriApePer':'PRIMER APELLIDO',
                                  'DatInmSegApePer':'SEGUNDO APELLIDO',
                                  'Genero':'GENERO',
                                  'DatInmNumDoc':'NUMERO AFILIADO',
                                  'DatInmTipDoc':'TIPO DOC AFILIADO',
                                  'dir_res_':'DIR_RESIDENCIA',
                                  'DIR':'DIR_RESIDENCIA_limpiada',
                                  'PerFecReg':'Fecha_vacuna',
                                  'PerHorReg':'Hora_vacuna',
                                  'DatInmFecNa':'Fecha_Nacimiento',
                                  'DatInmTelPer':'Celular',
                                  'DatInmCorPer':'Correo',
                                  'DatInmMun':'Municipio',
                                  'DatInmDep':'Departamento',
                                  'DatInmAcepto':'Acepta_inmunizacion',
                                  'DatInmAut':'Autoriza_inmunizacion',
                                  'DatInmCon':'Consentimiento_inmunizacion'}, inplace=True)

faltantes_final

,IdPersona,TIPO DOC AFILIADO,NUMERO AFILIADO,PRIMER NOMBRE,SEGUNDO NOMBRE,PRIMER APELLIDO,SEGUNDO APELLIDO,Fecha_Nacimiento,DIR_RESIDENCIA,Departamento,Municipio,Celular,Correo,Autoriza_inmunizacion,DatInmAcep,Consentimiento_inmunizacion,Acepta_inmunizacion,Fecha_vacuna,Hora_vacuna,GENERO,Comuna,DIR_RESIDENCIA_limpiada,Nombcomuna,numcomuna,Barrio,LONGITUD,LATITUD
0,137,CE,1043876663,JAINNE,NaN,CABALLERO,ACUÃ‘A,1996-12-08,CALLE 10 # - 27-67 OTRO BUCARAMANGA - SANTANDER,SANTANDER,BUCARAMANGA,31655894420,hinata0896@hotmail.com,SI,SI,SI,SI,2021-02-19,13:21:09,F,03. SAN FRANCISCO,calle 10 27 67 otro bucaramanga santander,SAN FRANCISCO,3,UNIVERSIDAD,-73.120004,7.137728
1,529,CE,446476,RAFAEL,DE JESUS,BRICEÃ‘O,CARRERO,1995-03-16,CALLE 20 # 30- 27 SAN ALONSO BUCARAMANGA - S...,SANTANDER,BUCARAMANGA,3144760168,rbricenoc16@gmail.com,SI,SI,SI,SI,2021-02-21,07:45:33,M,13. ORIENTAL,calle 20 30 27 san alonso bucaramanga santander,ORIENTAL,13,SAN ALONSO,-73.116820,7.130270
2,915,CC,718023,ROBERTO,CARLO,CORREA,BASTIDAS,1968-09-02,CARRERA 36 # - 37-26 APTO 604 EL PRADO BUCAR...,SANTANDER,BUCARAMANGA,3324374537,rcorrea93@gmail.com,SI,SI,SI,SI,2021-02-22,19:36:25,M,13. ORIENTAL,carrera 36 37 26 el prado bucaramanga santander,ORIENTAL,13,EL PRADO,-73.110000,7.123370
3,951,CC,476188,KATYNA,GABRIELA,DI LIZIO,MIELE,1983-07-21,CARRERA 39- # - 46-79 CABECERA DEL LLANO BUC...,SANTANDER,BUCARAMANGA,3108550959,kgdm@hotmail.com,SI,SI,SI,SI,2021-02-23,07:20:47,F,12. CABECERA DEL LLANO,carrera 39 46 79 cabecera del llano bucaramang...,CABECERA DEL LLANO,12,CABECERA DEL LLANO,-73.107170,7.117780


In [73]:
faltantes_final.shape

(4, 27)

In [74]:
merged.shape

(555182, 43)

In [77]:
for col in list(set(vacunacion.keys()).difference(set(faltantes_final.keys()))):
    print(col)
    #cruce_no_enc[col] = np.nan
#merged = pd.concat([merged, cruce_no_enc])    

Fase vacunación
IPS 1
IPS 3
condicion_final
Edad
grupo etario
Profesion 3
Profesion 2
IPS 4
regimen
inmunizado
Area COVID
linea salud
PERFIL PROFESIONAL
IPS 2
Profesion 1
Dedicación
Profesion 4


In [78]:
del faltantes_final['IdPersona']
del faltantes_final['DatInmAcep']

In [80]:
merged = pd.concat([merged, faltantes_final]) 

In [81]:
merged['Fecha_Nacimiento'] = merged['Fecha_Nacimiento'].astype('datetime64[ns]')
merged['Fecha_vacuna'] = merged['Fecha_vacuna'].astype('datetime64[ns]')

In [82]:
merged.loc[~(merged['Fecha_Nacimiento'].isnull()) & 
            (merged.Edad.isnull()), 'Edad'] = np.floor((merged.loc[~(merged['Fecha_Nacimiento'].isnull()) & 
                                                       (merged.Edad.isnull()), 'Fecha_vacuna'] - merged.loc[~(merged['Fecha_Nacimiento'].isnull()) & 
                                                       (merged.Edad.isnull()), 'Fecha_Nacimiento']).dt.days/365.25)

In [83]:
query = (merged['grupo etario'].isnull()) & ~(merged.Edad.isnull())
merged.loc[query, 'grupo etario'] = merged.loc[query, 'Edad'].apply(lambda x: to_grupo_etario(x))

In [87]:
merged.loc[(~merged.Fecha_vacuna.isnull()), 'inmunizado'] = 'si'

In [88]:
merged.loc[merged.inmunizado=='si'].shape

(964, 43)

In [86]:
merged

,NUMERO AFILIADO,TIPO DOC AFILIADO,GENERO,Edad,grupo etario,Barrio,numcomuna,Nombcomuna,Comuna,LONGITUD,LATITUD,Fase vacunación,linea salud,regimen,IPS 1,IPS 2,Profesion 1,Profesion 2,PRIMER APELLIDO,SEGUNDO APELLIDO,PRIMER NOMBRE,SEGUNDO NOMBRE,PERFIL PROFESIONAL,Area COVID,Dedicación,Correo,Celular,Municipio,Fecha_Nacimiento,Departamento,DIR_RESIDENCIA,DIR_RESIDENCIA_limpiada,Acepta_inmunizacion,Autoriza_inmunizacion,Consentimiento_inmunizacion,Fecha_vacuna,Hora_vacuna,inmunizado,IPS 3,IPS 4,Profesion 3,Profesion 4,condicion_final
0,6612272,CC,M,65.0,60 a 69 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 2,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,albarracin,NaN,luis,alfonso,NaN,NaN,NaN,NaN,3.158410e+09,NaN,1955-11-14,NaN,av cl 27 47,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
1,1005339195,CC,M,20.0,20 a 29 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.136905,7.155997,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
2,63303324,CC,F,58.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
3,37930938,CC,F,55.0,50 a 59 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
4,63447746,CC,F,49.0,40 a 49 años,ALTOS DEL KENNEDY,1.0,NORTE,01. NORTE,-73.137180,7.156320,Etapa 3,NaN,Régimen subsidiado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,no,NaN,NaN,NaN,NaN,vivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,10958787325,CC,M,35.0,30 a 39 años,MEJORAS PUBLICAS,13.0,ORIENTAL,13. ORIENTAL,-73.114180,7.120000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLANCO,MANTILLA,ANGEL,ROBERTO,NaN,NaN,NaN,pegasos1985@hotmail.com,3.118940e+09,BUCARAMANGA,1985-09-12,SANTANDER,CARRERA 28 # - 41-16 MEJORAS PUBLICAS BUCARA...,carrera 28 41 16 mejoras publicas bucaramanga ...,SI,SI,SI,2021-02-23,07:35:13,si,NaN,NaN,NaN,NaN,NaN
0,1043876663,CE,F,24.0,20 a 29 años,UNIVERSIDAD,3.0,SAN FRANCISCO,03. SAN FRANCISCO,-73.120004,7.137728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CABALLERO,ACUÃ‘A,JAINNE,NaN,NaN,NaN,NaN,hinata0896@hotmail.com,3.165589e+10,BUCARAMANGA,1996-12-08,SANTANDER,CALLE 10 # - 27-67 OTRO BUCARAMANGA - SANTANDER,calle 10 27 67 otro bucaramanga santander,SI,SI,SI,2021-02-19,13:21:09,NaN,NaN,NaN,NaN,NaN,NaN
1,446476,CE,M,25.0,20 a 29 años,SAN ALONSO,13.0,ORIENTAL,13. ORIENTAL,-73.116820,7.130270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BRICEÃ‘O,CARRERO,RAFAEL,DE JESUS,NaN,NaN,NaN,rbricenoc16@gmail.com,3.144760e+09,BUCARAMANGA,1995-03-16,SANTANDER,CALLE 20 # 30- 27 SAN ALONSO BUCARAMANGA - S...,calle 20 30 27 san alonso bucaramanga santander,SI,SI,SI,2021-02-21,07:45:33,NaN,NaN,NaN,NaN,NaN,NaN
2,718023,CC,M,52.0,50 a 59 años,EL PRADO,13.0,ORIENTAL,13. ORIENTAL,-73.110000,7.123370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CORREA,BASTIDAS,ROBERTO,CARLO,NaN,NaN,NaN,rcorrea93@gmail.com,3.324375e+09,BUCARAMANGA,1968-09-02,SANTANDER,CARRERA 36 # - 37-26 APTO 604 EL PRADO BUCAR...,carrera 36 37 26 el prado bucaramanga santander,SI,SI,SI,2021-02-22,19:36:25,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 5.7MB/s 


In [91]:
merged.to_excel('BD_inm_principal_cod_2.xlsx', index=False, engine='xlsxwriter')

In [89]:
del vacunacion
del vacunacion_1
del vacunacion_2

In [92]:
merged.to_excel('/content/drive/MyDrive/Datos/Alcaldia/Analitica/vacunacion/BD_inm_principal_codificacion_2.xlsx', index=False, engine='xlsxwriter')